# Tugas Akhir
"Klasfikasi Perintah Bahasa Natural Menggunakan *Global Vectors for Word Representations* (GloVe), *Convolutional Neural Networks*, dan Teknik *Transfer Learning* Untuk Aplikasi *Chatbots*"<br>

Irfan Hanif<br>
5114100177

### Menyiapkan Source Dataset
Source Dataset akan digunakan untuk melatih CNN sebelum dilakukan proses Transfer Learning

In [2]:
import pandas as pd
import numpy as np

In [2]:
data_path = "data/data_source/"
filename = "lapor.csv"

In [3]:
train_df = pd.read_csv(data_path + filename, delimiter=',')
train_df.head()

,id,TrackingID,Pelapor,JudulLaporan,IsiLaporan,Kategori,KategoriID,DisposisiInstansi,DisposisiInstansiID,Area,...,TanggalDisposisi,TanggalLaporanMasuk,TanggalLaporanAktivitasTerakhir,TanggalLaporanDitutup,TimestampLaporanMasuk,TimestampTanggalDisposisi,TimestampTanggalAktivitasTerakhir,TimestampTanggalLaporanDitutup,created_at,updated_at
0,1,22022,628214771xxxx,Tenaga Dokter di Nusa Tenggara Timur,Pak Menteri Kesehatan. Tolong perhatikan kami ...,Bidang Kesejahteraan Rakyat,16,Kementerian Kesehatan,118,Nusa Tenggara Timur,...,04 June 2012 11:24:06,28 May 2012 21:47:41,04 June 2012 11:24:06,27 July 2012 01:00:02,1338216461,1338783846,1338783846,1.343326e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
1,2,22033,628574613xxxx,Standarisasi Pornografi & Pornoaksi,"Kemenkumham, standarisasi minimal pornografi &...",Teknologi Informasi dan Komunikasi,13,Kementerian Hukum dan HAM,248,Nasional,...,13 July 2012 18:13:34,29 May 2012 10:14:24,03 September 2014 12:45:38,25 September 2012 01:00:01,1338261264,1342178014,1409723138,1.348510e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
2,3,22039,628214771xxxx,Biaya Operasi untuk Pasien Tidak Mampu,"Menkes RI yang terhormat, tolong inspeksi ke R...",Kesehatan,5,Kementerian Kesehatan,118,Nusa Tenggara Timur,...,04 June 2012 14:31:27,29 May 2012 11:41:25,22 January 2015 17:42:25,27 July 2012 01:00:02,1338266485,1338795087,1421923345,1.343326e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
3,4,22067,628139296xxxx,Pembayaran Gaji Dokter PTT di Maluku Utara,"Selamat pagi, saya Andrianto, dokter PTT di pe...",Kesehatan,5,Kementerian Kesehatan,118,Maluku Utara,...,30 May 2012 19:11:38,30 May 2012 09:30:47,04 January 2015 04:13:23,08 January 2013 01:00:02,1338345047,1338379898,1420319603,1.357582e+09,2015-12-04 02:08:00,2015-12-04 02:08:00
4,5,22145,628215460xxxx,Pembalakan Liar di Kalimantan Tengah,Yth. Bpk. Kapolda Kalimantan Tengah. Penegak h...,Lingkungan Hidup dan Penanggulangan Bencana,11,Badan Reserse Kriminal,865,Kalimantan Tengah,...,15 June 2012 16:20:36,31 May 2012 16:40:41,06 September 2015 06:59:50,25 June 2012 12:53:12,1338457241,1339752036,1441497590,1.340604e+09,2015-12-04 02:08:00,2015-12-04 02:08:00


In [4]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print kategori_total_list

Bantuan Langsung Sementara Masyarakat (BLSM)                                 14739
Topik Lainnya                                                                 9175
Infrastruktur                                                                 6949
Reformasi Birokrasi dan Tata Kelola                                           6840
Bantuan Siswa Miskin (BSM)                                                    6608
Beras Miskin (Raskin)                                                         5952
Kepesertaan Non-KPS                                                           4557
Pendidikan                                                                    3506
Lingkungan Hidup dan Penanggulangan Bencana                                   2748
Kartu Indonesia Pintar (KIP)                                                  2391
Kesehatan                                                                     2220
Kepesertaan KPS                                                               2036
Impl

In [5]:
kategori = train_df["Kategori"].values
for i in range(len(kategori)):
    kat = str(kategori[i]).lower()
    if "sehat" in kat or "obat" in kat or "jamkesmas" in kat:
        kategori[i] = "Kesehatan"
    elif "banjir" in kat or "bencana" in kat:
        kategori[i] = "Lingkungan Hidup dan Penanggulangan Bencana"
    elif "bbm" in kat:
        kategori[i] = "BBM"
    elif "perekonomian" in kat:
        kategori[i] = "Perekonomian"
    elif "usaha" in kat:
        kategori[i] = "Perdagangan, Perindustrian, Iklim Usaha, dan Investasi"
    elif "pendidikan" in kat:
        kategori[i] = "Pendidikan"
    elif "kewaspadaan" in kat or "perundungan" in kat or "kantor cabang" in kat or "sms" in kat or "haji" in kat or "pemberdayaan masyarakat" in kat or "situasi khusus" in kat:
        kategori[i] = "Topik Lainnya"

In [6]:
train_df["Kategori Edited"] = kategori

In [7]:
kategori_total_list = train_df["Kategori"].value_counts()
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print kategori_total_list

Bantuan Langsung Sementara Masyarakat (BLSM)                                14739
Topik Lainnya                                                                9218
Infrastruktur                                                                6949
Reformasi Birokrasi dan Tata Kelola                                          6840
Bantuan Siswa Miskin (BSM)                                                   6608
Beras Miskin (Raskin)                                                        5952
Kepesertaan Non-KPS                                                          4557
Pendidikan                                                                   3892
Kesehatan                                                                    3615
Lingkungan Hidup dan Penanggulangan Bencana                                  3129
Kartu Indonesia Pintar (KIP)                                                 2391
Kepesertaan KPS                                                              2036
BBM             

In [8]:
laporan = train_df["IsiLaporan"].values
laporan

array([ 'Pak Menteri Kesehatan. Tolong perhatikan kami RSU di daerah NTT, tenaga Dokter kurang & pelayanan tidak maksimal, tapi bisa melayani 3 s/d 4 tempat. Sangat berbahaya... Trims.',
       'Kemenkumham, standarisasi minimal pornografi & pornoaksi tolong dibuat dan disosialisasikan kepada masyarakat.',
       'Menkes RI yang terhormat, tolong inspeksi ke RSUD WZ. Yohanis, Kupang. Jika pasien tidak mampu, operasi penyakit dalam diminta bayar sekitar 10 juta, meskipun ada keterangan tidak mampu. Apakah ini benar?',
       ...,
       'Lstrik di ribuan pelanggan PLN Desa Brabe Kec. Maron Kab. Probolinggo, Jatim, Wilayah Kerja PLN Kraksaan Kab. Prob Jatim sering padam berjam-jam, antara lain tgl 15,24,25 September, 5,13 Oktober,9,10,11,13,16,30 November,1 Desember padam dan saat ini 3 De',
       'Kepada yth pengurus BPJSTK, saya mohon bantuannya, waktu saya daftar pertama kali menggunakan nmr HP 089601729737, namun sekarang no tsb b sudah tidak aktip, ketika saya ingin cek saldo bpjs 

In [10]:
def getWordEmbedding(word, cursor):
#     word = word.replace("'", "''")
    sql = """select vec from term where term like %s"""
    cursor.execute(sql, (str(word),))
    data = cursor.fetchall()
    if len(data) > 0:
        decoded_vec = json.JSONDecoder().decode(data[0][0])
        vec = np.asarray(decoded_vec, dtype=np.float32)
        return True, vec
    else:
        return False, data
    
def myTokenizer(content, lower=True):
    raw = content.split(' ')
    remover = re.compile("[^a-zA-Z-]")
    
    token = []
    
    for i in raw:
        term = remover.sub('', i)
        if lower == True:
            term = term.lower()
        token.append(term)
    tokenized = filter(None, token)
    
    return tokenized

def sentenceToVec(string):
    if type(string) is not str:
        return np.zeros((300,), dtype=float)
    
    string = string.replace('\n', '')
    string = np.array(myTokenizer(string))
    
    feature = None
    begin = True
    for word in string:
        stat, vec = getWordEmbedding(word, cursor)
        if not stat:
            continue
        if begin:
            begin = False
            feature = vec
        else:
            feature += vec
            # feature = np.concatenate([feature, vec])
    
    if feature is not None:
        feature = feature/np.linalg.norm(feature)
    else:
        feature = np.zeros((300,), dtype=float)
    
    return feature

import re
import json
import unicodedata
import mysql.connector
from tqdm import tqdm

db = mysql.connector.connect(user="root", password='namamuran', database="glove-300")
cursor = db.cursor(buffered=True)

pbar = tqdm(total=len(laporan))
ftr = []
for i in laporan:
    try:
        ftr.append(sentenceToVec(i))
    except Exception as err:
        print err
        print i
        break
    pbar.update(1)
pbar.close()
ftr = np.array(ftr)

100%|██████████| 87766/87766 [16:28<00:00, 88.81it/s] 


In [11]:
nan_idx = []
for i in range(len(kategori)):
    if kategori[i] is np.nan:
        nan_idx.append(i)
kategori_c = np.delete(kategori, nan_idx)
ftr_c = np.delete(ftr, nan_idx, 0)

In [12]:
cls = []
kategori_u = pd.unique(kategori_c)
kategori_u = kategori_u.tolist()
for i in range(len(kategori_c)):
    one_hot = np.zeros((len(kategori_u),), dtype=int)
    idx = kategori_u.index(kategori_c[i])
    one_hot[idx] = 1
    cls.append(one_hot)
cls = np.array(cls)

print "Mengecek array size dari fitur dan kelas..."
print "size of ftr " + str(ftr_c.shape)
print "size of cls " + str(cls.shape)

Mengecek array size dari fitur dan kelas...
size of ftr (87602, 300)
size of cls (87602, 68)


In [13]:
np.save(data_path + "ftr-data-source.npy", ftr_c)
np.save(data_path + "cls-data-source.npy", cls)

## SMOTE
Source Dataset ternyata tidak imbang antara satu kelas dan kelas lainnya. Maka, digunakan algoritma **SMOTE** untuk membuat jumlah data dan setiap kelasnya imbang (*balanced*).

In [14]:
ftr = np.load(data_path + "ftr-data-source.npy")
cls = np.load(data_path + "cls-data-source.npy")

In [15]:
indexed_cls = map(lambda x: np.argmax(x), cls)
pd.unique(indexed_cls)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67])

In [16]:
def valuesCount(arr):
    cls_dict = {}
    for i in arr:
        if i in cls_dict:
            cls_dict[i] += 1
        else:
            cls_dict[i] = 1
    return cls_dict

In [17]:
valuesCount(indexed_cls)

{0: 980,
 1: 428,
 2: 3615,
 3: 3129,
 4: 3892,
 5: 6840,
 6: 545,
 7: 218,
 8: 876,
 9: 720,
 10: 73,
 11: 305,
 12: 9218,
 13: 220,
 14: 40,
 15: 6949,
 16: 120,
 17: 36,
 18: 1134,
 19: 350,
 20: 642,
 21: 5952,
 22: 2029,
 23: 4557,
 24: 14739,
 25: 2036,
 26: 6608,
 27: 47,
 28: 317,
 29: 290,
 30: 399,
 31: 420,
 32: 363,
 33: 593,
 34: 100,
 35: 860,
 36: 114,
 37: 27,
 38: 53,
 39: 2391,
 40: 589,
 41: 370,
 42: 553,
 43: 340,
 44: 743,
 45: 342,
 46: 180,
 47: 100,
 48: 25,
 49: 16,
 50: 195,
 51: 41,
 52: 165,
 53: 16,
 54: 148,
 55: 51,
 56: 399,
 57: 177,
 58: 22,
 59: 37,
 60: 36,
 61: 220,
 62: 46,
 63: 104,
 64: 12,
 65: 61,
 66: 65,
 67: 324}

In [18]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
ftr_res, cls_res = sm.fit_sample(ftr, indexed_cls)

In [19]:
valuesCount(cls_res)

{0: 14739,
 1: 14739,
 2: 14739,
 3: 14739,
 4: 14739,
 5: 14739,
 6: 14739,
 7: 14739,
 8: 14739,
 9: 14739,
 10: 14739,
 11: 14739,
 12: 14739,
 13: 14739,
 14: 14739,
 15: 14739,
 16: 14739,
 17: 14739,
 18: 14739,
 19: 14739,
 20: 14739,
 21: 14739,
 22: 14739,
 23: 14739,
 24: 14739,
 25: 14739,
 26: 14739,
 27: 14739,
 28: 14739,
 29: 14739,
 30: 14739,
 31: 14739,
 32: 14739,
 33: 14739,
 34: 14739,
 35: 14739,
 36: 14739,
 37: 14739,
 38: 14739,
 39: 14739,
 40: 14739,
 41: 14739,
 42: 14739,
 43: 14739,
 44: 14739,
 45: 14739,
 46: 14739,
 47: 14739,
 48: 14739,
 49: 14739,
 50: 14739,
 51: 14739,
 52: 14739,
 53: 14739,
 54: 14739,
 55: 14739,
 56: 14739,
 57: 14739,
 58: 14739,
 59: 14739,
 60: 14739,
 61: 14739,
 62: 14739,
 63: 14739,
 64: 14739,
 65: 14739,
 66: 14739,
 67: 14739}

In [20]:
def convToOneHot(arr):
    total_cls = len(pd.unique(arr))
    cls = []
    for i in arr:
        temp_vec = np.zeros((total_cls,), dtype=int)
        temp_vec[i] = 1
        cls.append(temp_vec)
    cls = np.array(cls)
    return cls

In [21]:
cls_fin = convToOneHot(cls_res)
print "Panjang array kelas seletah convToOneHot: " + str(len(cls_fin[0]))

Panjang array kelas seletah convToOneHot: 68


In [22]:
np.save(data_path + "ftr_data_source_balanced.npy", ftr_res)
np.save(data_path + "cls_data_source_balanced.npy", cls_fin)

## Menyiapkan Target Dataset
Menyiapkan Target Dataset, mengubah bentuknya agar bisa menjadi masukan untuk CNN.

In [23]:
import re
import json
import unicodedata
import mysql.connector
import numpy as np
import pandas as pd

In [7]:
data_target_path = "data/data_target/"
filename_data_target = "data_target_schematics.csv"

In [30]:
def arrangeInputMatrix(data, labels):
    ftr_list = []
    cls_list = []
    
    unique_labels = pd.unique(labels)
    total_labels = len(unique_labels)
    
    for i in range(len(data)):
        cls = np.zeros((total_labels,), dtype=int)
        cls_idx = np.where(unique_labels == labels[i])[0][0]
        cls[cls_idx] = 1
        
        string = data[i].replace('\n', '')
        string = np.array(myTokenizer(string))

        begin = True
        for word in string:
            stat, vec = getWordEmbedding(word, cursor)
            if not stat:
                continue
            if begin:
                begin = False
                feature = vec
            else:
                feature += vec
                # feature = np.concatenate([feature, vec])

        feature = feature/np.linalg.norm(feature)
        ftr_list.append(feature)
        cls_list.append(cls)
        
    return np.array(ftr_list), np.array(cls_list)

In [31]:
print "convert into numpy array"
df = pd.read_csv(data_target_path + filename_data_target, sep=',', names=["documents", "labels"])
documents = df["documents"].values
labels = df["labels"].values

convert into numpy array


In [32]:
ftr, cls = arrangeInputMatrix(documents, labels)

In [33]:
print "checking len of array"
print len(documents) == len(ftr)
print len(documents) == len(cls)

print
print "checking size of arrays"
print "size of ftr: " + str(ftr.shape)
print "size pf cls: " + str(cls.shape)

checking len of array
True
True

checking size of arrays
size of ftr: (462, 300)
size pf cls: (462, 22)


In [35]:
np.save(data_target_path + "ftr_data_target.npy", ftr)
np.save(data_target_path + "cls_data_target.npy", cls)

## Splitting Dataset
Memisahkan Source Dataset dan Target Dataset menjadi Train Set dan Test Tes

## Splitting Target Dataset

In [8]:
ftr_data_target = np.load(data_target_path + "ftr_data_target.npy")
cls_data_target = np.load(data_target_path + "cls_data_target.npy")

In [9]:
data_target_path_splitted = "data/splitted/data_target/"

In [10]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.5, random_state=0)
for train_index, test_index in sss.split(ftr_data_target, cls_data_target):
    X, X_test = ftr_data_target[train_index], ftr_data_target[test_index]
    Y, Y_test = cls_data_target[train_index], cls_data_target[test_index]
    break
# X = np.expand_dims(X, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

In [11]:
np.save(data_target_path_splitted + "X.npy", X)
np.save(data_target_path_splitted + "Y.npy", Y)
np.save(data_target_path_splitted + "X_test.npy", X_test)
np.save(data_target_path_splitted + "Y_test.npy", Y_test)

## Splitting Source Dataset

In [3]:
data_source_path = "data/data_source/"

ftr_data_source = np.load(data_source_path + "ftr_data_source_balanced.npy")
cls_data_source = np.load(data_source_path + "cls_data_source_balanced.npy")

In [4]:
data_source_path_splitted = "data/splitted/data_source/"

In [5]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=3, test_size=0.1, random_state=0)
for train_index, test_index in sss.split(ftr_data_source, cls_data_source):
    X, X_test = ftr_data_source[train_index], ftr_data_source[test_index]
    Y, Y_test = cls_data_source[train_index], cls_data_source[test_index]
    break
# X = np.expand_dims(X, axis=2)
# X_test = np.expand_dims(X_test, axis=2)

In [6]:
np.save(data_source_path_splitted + "X.npy", X)
np.save(data_source_path_splitted + "Y.npy", Y)
np.save(data_source_path_splitted + "X_test.npy", X_test)
np.save(data_source_path_splitted + "Y_test.npy", Y_test)